In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import time as tim

In [2]:
def Nd(x):
    
    return 0.5*(1 + math.erf(x/math.sqrt(2)))

def BS(S, T, E, r, sigma):
    d1 = (math.log(S/E) + (r + 0.5*sigma**2)*T)/(sigma*math.sqrt(T))
    d2 = d1 - sigma*math.sqrt(T)
    N1 = 0.5*(1 + math.erf(d1/math.sqrt(2)))
    N2 = 0.5*(1 + math.erf(d2/math.sqrt(2)))
    
    return S*N1 - E*math.exp(-r*T)*N2 

def VarBS(S, T, K, r, sig):
    
    d = (1/sig*np.sqrt(T))*np.log(K*np.exp(-r*T)/S) + 0.5*sig*np.sqrt(T)
    
    N1 = 0.5*(1 + math.erf((-d + 2*sig*np.sqrt(T))/math.sqrt(2)))
    N2 = 0.5*(1 + math.erf((-d + sig*np.sqrt(T))/math.sqrt(2)))
    N3 = 0.5*(1 + math.erf(-d/math.sqrt(2)))
    
    return (S**2)*np.exp((sig**2)*T)*N1 - 2*K*S*np.exp(-r*T)*N2 + (K**2)*np.exp(-2*r*T)*N3 - (BS(S, T, K, r, sig))**2

def ExpTau(T, lam):
    
    return (1 - np.exp(-lam*T) - lam*T*np.exp(-lam*T))/(1 - np.exp(-lam*T)) + T*np.exp(-lam*T)

In [4]:
#Parameters of MC estimation:

S_0 = 100. #Initial stock price
r = 0.05 #Risk-free rate
sig = 0.2 #Stock price volatility
K = 100. #Strike price
lam = 0.05 #Intensity of departure process

T = 10. #Time to expiry
N = 2**10 #Number of sub-intervals of [0,T]
dt = T/float(N) #Length of each sub-interval

M = 10000 #Number of Monte Carlo samples
c = -1 #Parameter c of control variate

#############################################

MC = np.zeros(M)
MC_c = np.zeros(M)

t1 = tim.time()

E_tau = ExpTau(T, lam)

for m in range(0, M):
    
    Z = np.random.normal(0, 1, N)

    S = np.zeros(N)
    S[0] = S_0
    
    I = 0 #Indicator: 1 if departure occurs, 0 otherwise
    
    for i in range(0, N-1):
        
        S[i+1] = S[i]*np.exp( (r - 0.5*(sig**2))*dt + sig*np.sqrt(dt)*Z[i] )
        
    for j in range(0, N-1):
        
        P = np.random.poisson(lam*dt, 1)
        
        if P >= 1:
            I = 1
            t = (j+1)*dt
            S_t = S[j+1]
            break
    
    if I == 1:
        V = np.exp(-r*t)*np.maximum(S_t - K, 0)
    else:
        V = np.exp(-r*T)*np.maximum(S[N-1] - K, 0)

    MC[m] = V
    MC_c[m] = V + c*((np.exp(-r*E_tau))*np.maximum(S[int(E_tau/dt)-1] - K, 0) - BS(S_0, E_tau, K, r, sig))

t2 = tim.time()

#Brute-force MC estimate:

mean = np.mean(MC)
sd = np.std(MC)
CI = [round(mean - 1.96*sd/np.sqrt(M), 4), round(mean + 1.96*sd/np.sqrt(M), 4)]

#Control variate:

mean_c = np.mean(MC_c)
sd_c = np.std(MC_c)
CI_c = [round(mean_c - 1.96*sd_c/np.sqrt(M), 4), round(mean_c + 1.96*sd_c/np.sqrt(M), 4)]

########################################################

print('Time taken:', round((t2 - t1), 4), 'seconds.')
print('Brute force MC estimate:', round(mean, 4), ',', CI, 'with sampling error:', round(sd, 4))
print('MC estimate with control variate:', round(mean_c, 4), ',', CI_c, 'with sampling error:', round(sd_c, 4))

Time taken: 57.2027 seconds.
Brute force MC estimate: 38.7459 , [37.5695, 39.9224] with sampling error: 60.0222
MC estimate with control variate: 38.7022 , [37.8522, 39.5522] with sampling error: 43.3681
